# GBasis Tutorial - Basis Sets

# 1 Basis Sets

In [`gbasis`](http://gbasis.qcdevs.org/intro.html), a basis is defined as a list of [`GeneralizedContractionShell`](http://gbasis.qcdevs.org/_autosummary/gbasis.html#module-gbasis.contractions) objects, each one representing a linear combination of primitives and containing all the information defining this combination (i.e the angular momentum of the shell, the center of the shell, the exponents of the primitives, the contraction coefficients, and the normalization coefficients).

This notebook showcases the different ways to define a basis set for a molecule and how to transform them.

## 1.1 Loading Basis Sets

Basis set information is usually available in text format, such as from the [Basis Set Exchange](https://www.basissetexchange.org/). In `gbasis`, two of the most popular formats are supported:
  - Gaussian94 (.gbs)
  - NWChem (.nw).

The following example shows how to load a basis set from a Gaussian94  or a NWChem file.

### 1.1.1 Gaussian94 Format

In [1]:
import numpy as np
from gbasis.parsers import parse_gbs, parse_nwchem

# load hydrogen atom def2-SVP basis set information with the Gaussian94 format
gbs_basis_dict = parse_gbs("hydrogen_def2-svp.1.gbs")

# basis set information is stored as
# {'Atom Symbol': [(Angular Momentum, [Exponents], [Coefficients]), ...]}
print("def2-SVP Basis Set Loaded from Gaussian94 Format:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom}")
    print(f"   Number of shells: {len(gbs_basis_dict[atom])}")
    for shell_num, shell in enumerate(gbs_basis_dict[atom]):
        print(f"   Shell {shell_num} has angular momentum {shell[0]}")
        print(f"   Shell {shell_num} has exponents {shell[1]}")
        print(f"   Shell {shell_num} has coefficients {shell[2].flatten()}")


def2-SVP Basis Set Loaded from Gaussian94 Format:
Atom: H
   Number of shells: 3
   Shell 0 has angular momentum 0
   Shell 0 has exponents [13.010701    1.9622572   0.44453796]
   Shell 0 has coefficients [0.01968216 0.13796524 0.47831935]
   Shell 1 has angular momentum 0
   Shell 1 has exponents [0.12194962]
   Shell 1 has coefficients [1.]
   Shell 2 has angular momentum 1
   Shell 2 has exponents [0.8]
   Shell 2 has coefficients [1.]


### 1.1.2 NWChem Format

In [2]:
print("Loading def2-SVP Basis Set from NWChem Format...")
nw_basis_dict = parse_nwchem("hydrogen_def2-svp.1.nw")

print("Checking that the Gaussian94 and NWChem loaded basis sets are the same:")
print("Both loaded basis sets have the same:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom} is the same on both cases: {atom in nw_basis_dict}")
    print(f"Number of shells is the same: {len(gbs_basis_dict[atom]) == len(nw_basis_dict[atom])}")
    # check that the angular momentum, exponents, and coefficients are the same for each shell
    contractions_pair = enumerate(zip(gbs_basis_dict[atom], nw_basis_dict[atom]))
    for shell_num, (gb_shell, nw_shell) in contractions_pair:
        print(
            f"Shell {shell_num} has the same angular momentum: {gb_shell[0] == nw_shell[0]}"
        )
        print(
            f"Shell {shell_num} has the same exponents: {np.allclose(gb_shell[1], nw_shell[1])}"
        )
        print(
            f"Shell {shell_num} has the same coefficients: {np.allclose(gb_shell[2], nw_shell[2])}"
        )

Loading def2-SVP Basis Set from NWChem Format...
Checking that the Gaussian94 and NWChem loaded basis sets are the same:
Both loaded basis sets have the same:
Atom: H is the same on both cases: True
Number of shells is the same: True
Shell 0 has the same angular momentum: True
Shell 0 has the same exponents: True
Shell 0 has the same coefficients: True
Shell 1 has the same angular momentum: True
Shell 1 has the same exponents: True
Shell 1 has the same coefficients: True
Shell 2 has the same angular momentum: True
Shell 2 has the same exponents: True
Shell 2 has the same coefficients: True


## 1.2 Building Basis Instances

Once the basis set data for a list of atoms is loaded, `gbasis` allows users to build specific contraction shells for any molecule composed of these atoms. This is achieved with the `make_contraction_shells` function of the [`parsers`](http://gbasis.qcdevs.org/_autosummary/gbasis.html#module-gbasis.parsers) module. The coordinate system used by each contraction shell must be specified by the user. The following example shows how to use this function to build different contraction shells for $\mathrm{H}_{2}$.

### 1.2.1 Cartesian Contraction Shells
To build the contraction shells in the Cartesian coordinate system, the `make_contraction_shells` function must be called with `coord_types="cartesian"` or `coord_types="c"`. The following example shows how to build the contraction shells for $\mathrm{H}_{2}$ in the Cartesian coordinate system.

In [3]:
from gbasis.parsers import make_contractions

# use H2 molecule as an example system
atoms = ["H", "H"]
atcoords = np.array([[0, 0, 0], [0, 0, 1]])

# make contractions for the hydrogen def2-SVP basis set
basis = make_contractions(gbs_basis_dict, atoms, atcoords, coord_types="cartesian")

print("Number of contracted basis functions:", len(basis))  # 3 for each H atom
print(f"Types of contractions: {[i.coord_type for i in basis]}", end="\n\n")
print("Showing first three contraction shells:")
for i, shell in enumerate(basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {shell.coord}")
    print(f"   Angular momentum: {shell.angmom}")
    print(f"   Primitive coefficients {shell.coeffs.T}")
    print(f"   Primitive exponents {shell.exps}")
    print(f"   Primitive normalization constant {shell.norm_cont}")

Number of contracted basis functions: 6
Types of contractions: ['cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian', 'cartesian']

Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.01968216 0.13796524 0.47831935]]
   Primitive exponents [13.010701    1.9622572   0.44453796]
   Primitive normalization constant [[1.70131166]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [0.12194962]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[1.]]
   Primitive exponents [0.8]
   Primitive normalization constant [[1. 1. 1.]]


### 1.2.2 Spherical Contraction Shells
To build the contraction shells in the spherical coordinate system, the `make_contraction_shells` function must be called with `coord_types="spherical"` or `coord_types="p"` (where `p` means "pure"). The following example shows how to build the contraction shells for $\mathrm{H}_{2}$ in the spherical coordinate system.

In [4]:
# make contractions for the hydrogen def2-SVP basis set
basis = make_contractions(gbs_basis_dict, atoms, atcoords, coord_types="spherical")

print("Number of contracted basis functions:", len(basis))  # 3 for each H atom
print(f"Types of contractions: {[i.coord_type for i in basis]}", end="\n\n")
print("Showing first three contraction shells:")
for i, shell in enumerate(basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {shell.coord}")
    print(f"   Angular momentum: {shell.angmom}")
    print(f"   Primitive coefficients {shell.coeffs.T}")
    print(f"   Primitive exponents {shell.exps}")
    print(f"   Primitive normalization constant {shell.norm_cont}")

Number of contracted basis functions: 6
Types of contractions: ['spherical', 'spherical', 'spherical', 'spherical', 'spherical', 'spherical']

Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.01968216 0.13796524 0.47831935]]
   Primitive exponents [13.010701    1.9622572   0.44453796]
   Primitive normalization constant [[1.70131166]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [0.12194962]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[1.]]
   Primitive exponents [0.8]
   Primitive normalization constant [[1. 1. 1.]]


### 1.2.3 Mixed Contraction Shell types
A coordinate system can be specified for each contraction shell individually, allowing for mixed contraction shell types. For this, a list of strings with values `"cartesian"`, `"c"` or `"spherical"`, `"p"` must be passed to the `coord_types` argument of the `make_contraction_shells` function. The length of this list must be equal to the number of contraction shells. The following example shows how to build the contraction shells for $\mathrm{H}_{2}$ with the first shell in the Cartesian coordinate system and the second shell in the spherical coordinate system.

In [5]:
# list of coordinate types, one for each shell
coord_types = ["c", "p", "spherical", "cartesian", "p", "cartesian"]

# make contractions for the hydrogen def2-SVP basis set
basis = make_contractions(gbs_basis_dict, atoms, atcoords, coord_types=coord_types)

print("Number of contracted basis functions:", len(basis))  # 3 for each H atom
print(f"Types of contractions: {[i.coord_type for i in basis]}", end="\n\n")
print("Showing first three contraction shells:")
for i, shell in enumerate(basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {shell.coord}")
    print(f"   Angular momentum: {shell.angmom}")
    print(f"   Primitive coefficients {shell.coeffs.T}")
    print(f"   Primitive exponents {shell.exps}")
    print(f"   Primitive normalization constant {shell.norm_cont}")

Number of contracted basis functions: 6
Types of contractions: ['cartesian', 'spherical', 'spherical', 'cartesian', 'spherical', 'cartesian']

Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.01968216 0.13796524 0.47831935]]
   Primitive exponents [13.010701    1.9622572   0.44453796]
   Primitive normalization constant [[1.70131166]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [0.12194962]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[1.]]
   Primitive exponents [0.8]
   Primitive normalization constant [[1. 1. 1.]]


## 1.3 Interfacing With Other Packages

`gbasis` has been designed to be easily interfaced the [`iodata`](https://iodata.readthedocs.io/en/latest/) package, which handles the inputs and outputs for different quantum chemistry formats, such as Gaussian formatted checkpoint files (.fchk) and AIM wavefunction files (.wfn and .wfx). Additionally, `gbasis` can be directly interfaced with [`pySCF`](https://pyscf.org/), a popular Python package for quantum chemistry calculations.

### 1.3.1 [`iodata`](https://iodata.readthedocs.io/en/latest/) Package
This is the preferred way to load basis information from computational chemistry calculations in `gbasis`. The following example shows how to load a basis set from a Gaussian formatted checkpoint using the `iodata` package. The same approach can be used for loading the basis information of other formats supported by `iodata`.

In [6]:
from iodata import load_one
from gbasis.wrappers import from_iodata

# load a basis from an fchk file (water at uwB97XD/def2-TZVPD LOT) using IOData
iodata_mol = load_one("water.fchk")
basis = from_iodata(iodata_mol)
print(basis[0].coord)


# print("Basis loaded from IOData:")
# print(basis)
# basis[0].gbs_basis

# print the basis set information
print("Basis loaded from IOData:")
print("Number of contracted basis functions:", len(basis))
print("Showing first three contraction shells:")
for i, shell in enumerate(basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {shell.coord}")
    print(f"   Angular momentum: {shell.angmom}")
    print(f"   Primitive coefficients {shell.coeffs.T}")
    print(f"   Primitive exponents {shell.exps}")
    print(f"   Primitive normalization constant {shell.norm_cont}")

[ 0.01418428  0.01049743 -0.00741906]
Basis loaded from IOData:
Number of contracted basis functions: 24
Showing first three contraction shells:
Contraction shell #0
   Center: [ 0.01418428  0.01049743 -0.00741906]
   Angular momentum: 0
   Primitive coefficients [[5.72273352e-04 4.43532335e-03 2.30201077e-02 9.28224907e-02
  2.93785000e-01 6.74016045e-01]]
   Primitive exponents [27032.3826     4052.38714     922.327227    261.24071      85.3546414
    31.0350352]
   Primitive normalization constant [[1.]]
Contraction shell #1
   Center: [ 0.01418428  0.01049743 -0.00741906]
   Angular momentum: 0
   Primitive coefficients [[0.63839937 0.39534587]]
   Primitive exponents [12.2608607  4.9987076]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [ 0.01418428  0.01049743 -0.00741906]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [1.17031082]
   Primitive normalization constant [[1.]]


### 1.3.2 [`pySCF`](https://pyscf.org/) Package
`gbasis`has a direct interface with `pySCF` objects. To showcase this feature, a `pySCF` `Mole` object is created and its `basis` are loaded using `gbasis`.

In [7]:
from pyscf import gto
from gbasis.wrappers import from_pyscf

molecule = '''O 0 0 0;
H 0 1 0;
H 0 0 1'''

basis_name = "sto-3g"

print("Constructing pySCF molecule:")
print("Atomic coordinates:")
print(molecule)
print(f"Basis set used: {basis_name}")

# build an STO-3G basis for water using PySCF
pyscf_mol = gto.Mole()
pyscf_mol.build(
    atom = molecule,
    basis = basis_name,
)

print("Importing basis set from PySCF to gbasis:")
basis = from_pyscf(pyscf_mol)

print("Number of contracted basis functions:", len(basis))
print("Showing first three contraction shells:")
for i, shell in enumerate(basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {shell.coord}")
    print(f"   Angular momentum: {shell.angmom}")
    print(f"   Primitive coefficients {shell.coeffs.T}")
    print(f"   Primitive exponents {shell.exps}")
    print(f"   Primitive normalization constant {shell.norm_cont}")

Constructing pySCF molecule:
Atomic coordinates:
O 0 0 0;
H 0 1 0;
H 0 0 1
Basis set used: sto-3g
Importing basis set from PySCF to gbasis:
Number of contracted basis functions: 5
Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.15432897 0.53532814 0.44463454]]
   Primitive exponents [130.70932    23.808861    6.4436083]
   Primitive normalization constant [[0.99999999]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[-0.09996723  0.39951283  0.70011547]]
   Primitive exponents [5.0331513 1.1695961 0.380389 ]
   Primitive normalization constant [[0.99999999]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[0.15591627 0.60768372 0.39195739]]
   Primitive exponents [5.0331513 1.1695961 0.380389 ]
   Primitive normalization constant [[0.99999999 0.99999999 0.99999999]]
